In [1]:
import os
import sys
from glob import glob

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import desispec
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

from desiutil.log import get_logger, DEBUG

from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime
import heapq
import time
import copy
import numpy
from astropy.time import Time
from astropy.table import Table, vstack, unique, SortedArray
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython import display

In [2]:
#SkyPortal token:
secret_file = "/global/cfs/cdirs/desi/science/td/secrets/desidiff_sp.txt"
with open(secret_file, 'r') as file:
    token = file.read().replace('\n', '')
headers = {'Authorization': f'token {token}'}

filter_name = 'DESIDIFF'

all_plots_pdf = PdfPages("All_plots.pdf")

In [3]:
# read in and store in one place all the fibermap information in the spectra files

night = 20210608 
for current_filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-*.fits"):
    tile = current_filename.split('/')[-3]
    petal = current_filename.split('-')[1]
    ### daily_spectra, the precursor to current spectra, before coadding, to select night, unique target ids, and individual target ids
    ### spectra.select functionality will not work once coadded with coaddition.coadd_cameras
    daily_spectra = ((read_spectra(current_filename)).select(nights = night))

    table = Table.read(current_filename, format='fits',hdu=1, memmap=True) 
    ##### DAVE'S ADDITION ##############
    targetcols = [i for i in table.colnames if i[-7:] =='_TARGET']
    nonzerocheck = [True in k for k in [[j != 0 for j in table[targetcols][i]] for i in range(len(table))]]
    #a really ugly line, basically generates a list of bools, 
    #True if there is at least one nonzero element in all columns ending in _TARGET
    table.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
    #This gets the index of all False values from the previous list and removes those rows
    table = table['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
    ######## END DAVE'S ADDITION ############
    table = table[numpy.logical_and(table['OBJTYPE']=='TGT', table['FIBERSTATUS']==0)]


    for ref_filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
        print(ref_filename)
        try:
            if ref_filename.split('/')[-6] == 'fuji' or ref_filename.split('/')[-6] == 'guadalupe':
                prev_spectra = read_spectra(ref_filename)

                gen = (t for t in daily_spectra.target_ids()[numpy.where(daily_spectra.fibermap['OBJTYPE']=='TGT')] if t in prev_spectra.target_ids()[numpy.where(prev_spectra.fibermap['OBJTYPE']=='TGT')])
                for t in gen:
                    ### grab ra and dec values for use in SkyPortal functionality later
                    ra_dec = (table[table['TARGETID']==t]['TARGET_RA'][0], table[table['TARGETID']==t]['TARGET_DEC'][0])
                    ### match target ids
                    current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = t))
                    ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = t))

                    norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

                    # need to instantiate a Spectra object for the difference. 
                    ### Using 'dif_spectra = Spectra()' is bugging on dif_spectra.mask type=NoneType, can't assign.
                    #### dif_spectra = current_spectra
                    ### copy.deepcopy() is deprecated as memory expensive
                    dif_spectra = copy.deepcopy(current_spectra)
                    #### any problem with hardcoding in 'brz' for key in the following:
                    for key in (current_spectra.flux).keys():
                        current_spectra.flux[key] = current_spectra.flux[key]/norm
                        current_spectra.ivar[key] = current_spectra.ivar[key]*norm**2  
                        # subtraction of current and reference fluxes
                        dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                        ### couldn't dif_spectra.mask[key] == 2 by summing current.mask and reference.mask
                        # summation of current and reference masks
                        dif_spectra.mask[key] = current_spectra.mask[key] + ref_spectra.mask[key]
                        # inverted summation of current and spectra inverse variance
                        ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
                        dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])

                    # mean-subtracted difference
                    dif_flux_clipped = clipmean(dif_spectra.flux, dif_spectra.ivar, dif_spectra.mask)

                    # filters 
                    # Difference spectrum may have broadband signal
                    perband_filter = perband_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask)

                    # fractional increase
                    perband_inc = perband_increase(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ref_spectra.flux, ref_spectra.ivar, ref_spectra.mask)

                    # Difference spectrum may have high-frequency signal
                    perres_filter_broad = perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 100, nsig = 17)

                    perres_filter_narrow = perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 5, nsig = 10)

                    ### grab redrock data required for filters using redshift (zinfo)
                    current_filename = (current_filename.replace('spectra','redrock')).replace('fits', 'h5')

                    with h5py.File(current_filename, "r") as f:
                        zinfo = f['zfit']['t']['zfit'][0]['z']

                    # Search for signature lines of TDEs, only interested in Galaxies
                    linetable = line_finder(dif_spectra.wave, dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, zinfo)
                    # print(night, linetable)
                    Hline_score = Hline_filter(linetable)
                    # deriv_score = deriv_filter(dif_flux_clipped, dif_spectra.ivar,dif_spectra.mask)

                    #broadband
                    bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
                    narrowlinelogic = perres_filter_narrow >=2
                    broadlinelogic = perres_filter_broad >=3
                    # TDElogic = any([TDE_score == 2, TDE_score == 3, TDE_score == 4, TDE_score == 5])
                    Hlinelogic = any([Hline_score >= 1])
                    # derivlogic = any([deriv_score >= 3])

                    logic = [bblogic, narrowlinelogic,  broadlinelogic, Hlinelogic]
                    logic_name = ['Broadband', 'narrow line', 'broad line','Hline'] #must be in same order as logic!, use as mask
                    logic_name = numpy.ma.masked_array(logic_name, mask = [not i for i in logic])

                    plt.clf()
                    pdf_per_tile = PdfPages(str(tile)+".pdf")

                    plt.rcParams["figure.figsize"] = (20,6)
                    ####if any(logic):
                        #Uncomment next line if you want to print only those TargetIds that get plotted
                    #processed(t, tile, night)
                    fig = plt.figure()
                    #for b in dif_spectra.wave.keys():
                    ### have hardcoded 'brz' in place of for b in keys()...
                    for key in (current_spectra.flux).keys():
                        w=numpy.where(dif_spectra.mask[key][0] == 0)

                        plt.plot(dif_spectra.wave[key][w], dif_spectra.flux[key][0][w], color='red', label = 'Difference')
                        plt.plot(current_spectra.wave[key][w], current_spectra.flux[key][0][w], color='blue', alpha=0.5, label = 'New Spectrum')
                        plt.plot(ref_spectra.wave[key][w],ref_spectra.flux[key][0][w],color='green',alpha=0.5, label = 'Reference Spectrum')

                        plt.legend()
                        plt.xlabel('Wavelength (Å)')
                        plt.ylabel('Flux') 
                        plt.title(str(t) + "  " + str(night) + "  " + str(tile)  + "  " + str(logic_name))

                        plt.savefig(pdf_per_tile, format = 'pdf')
                        plt.savefig(all_plots_pdf, format = 'pdf')
                        plt.close()
                    """
                    #SkyPortal functionality begins:
                    #SkyPortal's Id for the object
                    objID = 'DESI{}'.format(str(t))

                    #Code to check if this object already exists in SkyPortal. If not, create it

                    response = requests.get("https://desi-skyportal.lbl.gov/api/candidates/{}".format(objID), headers={"Authorization": f"token {token}"})
                    if response.status_code == 400:
                        obj_data = {
                                "ra": ra_dec[0], #RA is required when creating a new object.
                                "dec": ra_dec[1], #Same for DEC
                                "id": objID,
                                "redshift": zinfo,
                                "filter_ids": [sp.filter_id(filter_name)],
                                "passed_at": str(datetime.datetime.utcnow()) #UTC time when the object passed the filter
                                }

                        response = requests.post(
                                "https://desi-skyportal.lbl.gov/api/candidates",
                                json = obj_data,
                                headers = {"Authorization": f"token {token}"})

                    #Now, we send the differenced spectrum to SkyPortal
                    #First, check if the same spectrum already exists under the object
                    response = requests.get("https://desi-skyportal.lbl.gov/api/sources/{}/spectra".format(objID),headers={"Authorization": f"token {token}"})
                    post = True
                    # for i in range (len(response.json()['data']['spectra'])):
                    #     if (wavelengths == response.json()['data']['spectra'][i]['wavelengths'] and flux == response.json()['data']['spectra'][i]['fluxes']):
                    #         post = False
                    #Only if the exact same spectrum doesn't already exist, upload it.
                    if post:
                        #Send difference spectrum to SP:
                        dif_spectrum_data = {
                                "obj_id": objID,
                                "wavelengths": dif_spectra.wave['brz'][w].tolist(),
                                "fluxes": dif_spectra.flux['brz'][0][w].tolist(),
                                "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                                "origin": "DESIDIFF_dif_spectrum", #Only the difference spectrum gets this tag in order to distinguish it on SkyPortal
                                "group_ids": [sp.group_id("DESI")],
                                "instrument_id": sp.instrument_id()
                                }
                        response = requests.post(
                                    '{}/api/spectrum'.format(sp.url),
                                    json= dif_spectrum_data,
                                    headers={"Authorization": f"token {token}"}) 

                        #Send new spectra to SP:
                        current_spectra_data = {
                            "obj_id": objID,
                            "wavelengths":current_spectra.wave['brz'][w].tolist(),
                            "fluxes": current_spectra.flux['brz'][0][w].tolist(),
                            "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                            "origin": "DESIDIFF",
                            "group_ids": [sp.group_id("DESI")],
                            "instrument_id": sp.instrument_id()
                            }
                        response = requests.post(
                                '{}/api/spectrum'.format(sp.url),
                                json= current_spectra_data,
                                headers={"Authorization": f"token {token}"}) 

                        #Code to find the ref_night that is closest to current_night to be used as 'observed_at' for SP
                        if ref_filename.split('/')[-6] == 'fuji':
                            ref_night = 20201214
                        elif ref_filename.split('/')[-6] == 'guadalupe':
                            ref_night = 20210514
                        current_night = datetime.datetime(int(str(night)[:4]), int(str(night)[4:6]), int(str(night)[6:]))
                        closest_night = datetime.datetime(int(str(ref_night)[:4]), int(str(ref_night)[4:6]), int(str(ref_night)[6:]))



                        ref_spectra_data = {
                            "obj_id": objID,
                            "wavelengths": ref_spectra.wave['brz'][w].tolist(),
                            "fluxes": ref_spectra.flux['brz'][0][w].tolist(),
                            "observed_at": str(closest_night),
                            "origin": "DESIDIFF",
                            "group_ids": [sp.group_id("DESI")],
                            "instrument_id": sp.instrument_id()
                            }
                        response = requests.post(
                                '{}/api/spectrum'.format(sp.url),
                                json= ref_spectra_data,
                                headers={"Authorization": f"token {token}"})    
                        """
                    all_plots_pdf.close()
                    pdf_per_tile.close()
        except:
            pass    

INFO:spectra.py:291:read_spectra: iotime 0.571 sec to read spectra-6-1097-thru20210608.fits at 2022-07-06T19:01:01.153731
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.444 sec to read coadd-6-1097-thru20210608.fits at 2022-07-06T19:01:03.449608


/tmp/ipykernel_22677/1769395560.py:56: RuntimeWarning: divide by zero encountered in true_divide
  dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])


INFO:spectra.py:291:read_spectra: iotime 0.511 sec to read spectra-4-1097-thru20210608.fits at 2022-07-06T19:01:04.769524
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-4-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-4-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-4-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.451 sec to read coadd-4-1097-thru20210608.fits at 2022-07-06T19:01:06.990467
INFO:spectra.py:291:read_spectra: iotime 0.511 sec to read spectra-8-1097-thru20210608.fits at 2022-07-06T19:01:13.952347
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-8-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-8-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-8-1097-thru20210608

/global/cscratch1/sd/clepart/timedomain/desidiff/src/scores.py:76: RuntimeWarning: divide by zero encountered in true_divide
  newston[b]=numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/ivar[b][0], conk, mode='valid')))


INFO:spectra.py:291:read_spectra: iotime 0.518 sec to read spectra-1-1097-thru20210608.fits at 2022-07-06T19:01:23.264997
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-1-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-1-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-1-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.445 sec to read coadd-1-1097-thru20210608.fits at 2022-07-06T19:01:25.392126
INFO:spectra.py:291:read_spectra: iotime 0.416 sec to read spectra-9-1097-thru20210608.fits at 2022-07-06T19:01:32.446629
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-9-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-9-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-9-1097-thru20210608

KeyboardInterrupt: 

In [ ]:
night_arr = getUnprocessedDates()

In [4]:
# read in and store in one place all the fibermap information in the spectra files
#night_arr = getUnprocessedDates()

for night in night_arr:   
    for current_filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-*.fits"):
        tile = current_filename.split('/')[-3]
        petal = current_filename.split('-')[1]
        ### daily_spectra, the precursor to current spectra, before coadding, to select night, unique target ids, and individual target ids
        ### spectra.select functionality will not work once coadded with coaddition.coadd_cameras
        try:
            daily_spectra = ((read_spectra(current_filename)).select(nights = night))
        except:
            daily_spectra = ((read_spectra(current_filename)).select(nights = (night-1)))

        table = Table.read(current_filename, format='fits',hdu=1, memmap=True) 
        ##### DAVE'S ADDITION ##############
        targetcols = [i for i in table.colnames if i[-7:] =='_TARGET']
        nonzerocheck = [True in k for k in [[j != 0 for j in table[targetcols][i]] for i in range(len(table))]]
        #a really ugly line, basically generates a list of bools, 
        #True if there is at least one nonzero element in all columns ending in _TARGET
        table.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
        #This gets the index of all False values from the previous list and removes those rows
        table = table['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
        ######## END DAVE'S ADDITION ############
        table = table[numpy.logical_and(table['OBJTYPE']=='TGT', table['FIBERSTATUS']==0)]


        for ref_filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
            print(ref_filename)
            try:
                if ref_filename.split('/')[-6] == 'fuji' or ref_filename.split('/')[-6] == 'guadalupe':
                    prev_spectra = read_spectra(ref_filename)

                    gen = (t for t in daily_spectra.target_ids()[numpy.where(daily_spectra.fibermap['OBJTYPE']=='TGT')] if t in prev_spectra.target_ids()[numpy.where(prev_spectra.fibermap['OBJTYPE']=='TGT')])
                    for t in gen:
                        print(t)
                        ### grab ra and dec values for use in SkyPortal functionality later
                        #ra_dec = (table[table['TARGETID']==t]['TARGET_RA'][0], table[table['TARGETID']==t]['TARGET_DEC'][0])
                        ra, dec = table['TARGETID' == t]['TARGET_RA'], table['TARGETID' == t]['TARGET_DEC']
                        ### match target ids
                        current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = t))
                        ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = t))
                           
                        print('coaddition.coadd_cameras completed')
                        norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

                        # need to instantiate a Spectra object for the difference. 
                        ### Using 'dif_spectra = Spectra()' is bugging on dif_spectra.mask type=NoneType, can't assign.
                        #### dif_spectra = current_spectra
                        ### copy.deepcopy() is deprecated as memory expensive
                        dif_spectra = copy.deepcopy(current_spectra)
                        #### any problem with hardcoding in 'brz' for key in the following:
                        for key in (current_spectra.flux).keys():
                            current_spectra.flux[key] = current_spectra.flux[key]/norm
                            current_spectra.ivar[key] = current_spectra.ivar[key]*norm**2 
                            print('normalization completed')
                            # subtraction of current and reference fluxes
                            dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                            print('flux subtraction completed')
                            ### couldn't dif_spectra.mask[key] == 2 by summing current.mask and reference.mask
                            # summation of current and reference masks
                            dif_spectra.mask[key] = current_spectra.mask[key] + ref_spectra.mask[key]
                            print('mask summation completed')
                            # inverted summation of current and spectra inverse variance
                            ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
                            dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])
                            print('ivar inverse summation completed')

                        # mean-subtracted difference
                        dif_flux_clipped = clipmean(dif_spectra.flux, dif_spectra.ivar, dif_spectra.mask)
                        print('clipmean completed')

                        # filters 
                        # Difference spectrum may have broadband signal
                        perband_filter = perband_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask)

                        # fractional increase
                        perband_inc = perband_increase(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ref_spectra.flux, ref_spectra.ivar, ref_spectra.mask)

                        # Difference spectrum may have high-frequency signal
                        perres_filter_broad = perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 100, nsig = 17)

                        perres_filter_narrow = perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 5, nsig = 10)
                        print('SN filters completed')
                        ### grab redrock data required for filters using redshift (zinfo)
                        current_filename = (current_filename.replace('spectra','redrock')).replace('fits', 'h5')

                        with h5py.File(current_filename, "r") as f:
                            zinfo = f['zfit'][str(t)]['zfit'][0]['z']

                        # Search for signature lines of TDEs, only interested in Galaxies
                        linetable = line_finder(dif_spectra.wave, dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, zinfo)
                        # print(night, linetable)
                        Hline_score = Hline_filter(linetable)
                        # deriv_score = deriv_filter(dif_flux_clipped, dif_spectra.ivar,dif_spectra.mask)
                        print('sig. line filters completed')
                        #broadband
                        bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
                        narrowlinelogic = perres_filter_narrow >=2
                        broadlinelogic = perres_filter_broad >=3
                        # TDElogic = any([TDE_score == 2, TDE_score == 3, TDE_score == 4, TDE_score == 5])
                        Hlinelogic = any([Hline_score >= 1])
                        # derivlogic = any([deriv_score >= 3])
                        print('broadband logic completed')
                        logic = [bblogic, narrowlinelogic,  broadlinelogic, Hlinelogic]
                        logic_name = ['Broadband', 'narrow line', 'broad line','Hline'] #must be in same order as logic!, use as mask
                        logic_name = numpy.ma.masked_array(logic_name, mask = [not i for i in logic])

                        plt.clf()
                        pdf_per_tile = PdfPages(str(tile)+".pdf")

                        plt.rcParams["figure.figsize"] = (20,6)
                        ####if any(logic):
                            #Uncomment next line if you want to print only those TargetIds that get plotted
                        #processed(t, tile, night)
                        fig = plt.figure()
                        for key in (current_spectra.flux).keys():
                            w=numpy.where(dif_spectra.mask[key][0] == 0)

                            plt.plot(dif_spectra.wave[key][w], dif_spectra.flux[key][0][w], color='red', label = 'Difference')
                            plt.plot(current_spectra.wave[key][w], current_spectra.flux[key][0][w], color='blue', alpha=0.5, label = 'New Spectrum')
                            plt.plot(ref_spectra.wave[key][w],ref_spectra.flux[key][0][w],color='green',alpha=0.5, label = 'Reference Spectrum')

                            plt.legend()
                            plt.xlabel('Wavelength (Å)')
                            plt.ylabel('Flux') 
                            plt.title(str(t) + "  " + str(night) + "  " + str(tile)  + "  " + str(logic_name))

                            plt.savefig(pdf_per_tile, format = 'pdf')
                            plt.savefig(all_plots_pdf, format = 'pdf')
                            plt.close()
                        """
                        #SkyPortal functionality begins:
                        #SkyPortal's Id for the object
                        objID = 'DESI{}'.format(str(t))

                        #Code to check if this object already exists in SkyPortal. If not, create it

                        response = requests.get("https://desi-skyportal.lbl.gov/api/candidates/{}".format(objID), headers={"Authorization": f"token {token}"})
                        if response.status_code == 400:
                            obj_data = {
                                    "ra": ra, #RA is required when creating a new object.
                                    "dec": dec, #Same for DEC
                                    "id": objID,
                                    "redshift": zinfo,
                                    "filter_ids": [sp.filter_id(filter_name)],
                                    "passed_at": str(datetime.datetime.utcnow()) #UTC time when the object passed the filter
                                    }

                            response = requests.post(
                                    "https://desi-skyportal.lbl.gov/api/candidates",
                                    json = obj_data,
                                    headers = {"Authorization": f"token {token}"})

                        #Now, we send the differenced spectrum to SkyPortal
                        #First, check if the same spectrum already exists under the object
                        response = requests.get("https://desi-skyportal.lbl.gov/api/sources/{}/spectra".format(objID),headers={"Authorization": f"token {token}"})
                        post = True
                        # for i in range (len(response.json()['data']['spectra'])):
                        #     if (wavelengths == response.json()['data']['spectra'][i]['wavelengths'] and flux == response.json()['data']['spectra'][i]['fluxes']):
                        #         post = False
                        #Only if the exact same spectrum doesn't already exist, upload it.
                        if post:
                            #Send difference spectrum to SP:
                            dif_spectrum_data = {
                                    "obj_id": objID,
                                    "wavelengths": dif_spectra.wave['brz'][w].tolist(),
                                    "fluxes": dif_spectra.flux['brz'][0][w].tolist(),
                                    "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                                    "origin": "DESIDIFF_dif_spectrum", #Only the difference spectrum gets this tag in order to distinguish it on SkyPortal
                                    "group_ids": [sp.group_id("DESI")],
                                    "instrument_id": sp.instrument_id()
                                    }
                            response = requests.post(
                                        '{}/api/spectrum'.format(sp.url),
                                        json= dif_spectrum_data,
                                        headers={"Authorization": f"token {token}"}) 

                            #Send new spectra to SP:
                            current_spectra_data = {
                                "obj_id": objID,
                                "wavelengths":current_spectra.wave['brz'][w].tolist(),
                                "fluxes": current_spectra.flux['brz'][0][w].tolist(),
                                "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                                "origin": "DESIDIFF",
                                "group_ids": [sp.group_id("DESI")],
                                "instrument_id": sp.instrument_id()
                                }
                            response = requests.post(
                                    '{}/api/spectrum'.format(sp.url),
                                    json= current_spectra_data,
                                    headers={"Authorization": f"token {token}"}) 

                            #Code to find the ref_night that is closest to current_night to be used as 'observed_at' for SP
                            if ref_filename.split('/')[-6] == 'fuji':
                                ref_night = 20201214
                            elif ref_filename.split('/')[-6] == 'guadalupe':
                                ref_night = 20210514
                            current_night = datetime.datetime(int(str(night)[:4]), int(str(night)[4:6]), int(str(night)[6:]))
                            closest_night = datetime.datetime(int(str(ref_night)[:4]), int(str(ref_night)[4:6]), int(str(ref_night)[6:]))



                            ref_spectra_data = {
                                "obj_id": objID,
                                "wavelengths": ref_spectra.wave['brz'][w].tolist(),
                                "fluxes": ref_spectra.flux['brz'][0][w].tolist(),
                                "observed_at": str(closest_night),
                                "origin": "DESIDIFF",
                                "group_ids": [sp.group_id("DESI")],
                                "instrument_id": sp.instrument_id()
                                }
                            response = requests.post(
                                    '{}/api/spectrum'.format(sp.url),
                                    json= ref_spectra_data,
                                    headers={"Authorization": f"token {token}"})    
                        """
                        all_plots_pdf.close()
                        pdf_per_tile.close()
            except:
                pass    

len(night_arr): 240
--- get unprocessed dates took:  161.48444390296936 seconds ---
INFO:spectra.py:291:read_spectra: iotime 0.524 sec to read spectra-5-21163-thru20210607.fits at 2022-07-06T19:04:52.702019
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/21163/20210607/coadd-5-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/21163/20210607/coadd-5-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/21163/20210607/coadd-5-21163-thru20210607.fits
INFO:spectra.py:291:read_spectra: iotime 0.463 sec to read coadd-5-21163-thru20210607.fits at 2022-07-06T19:04:55.028337


/tmp/ipykernel_22677/3915911600.py:57: RuntimeWarning: divide by zero encountered in true_divide
  dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])


INFO:spectra.py:291:read_spectra: iotime 0.525 sec to read spectra-4-21163-thru20210607.fits at 2022-07-06T19:05:02.440668
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/21163/20210607/coadd-4-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/21163/20210607/coadd-4-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/21163/20210607/coadd-4-21163-thru20210607.fits
INFO:spectra.py:291:read_spectra: iotime 0.528 sec to read coadd-4-21163-thru20210607.fits at 2022-07-06T19:05:04.908580
INFO:spectra.py:291:read_spectra: iotime 0.524 sec to read spectra-3-21163-thru20210607.fits at 2022-07-06T19:05:12.026366
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/21163/20210607/coadd-3-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/21163/20210607/coadd-3-21163-thru20210607.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/21163/20210607/coadd-3-211

RuntimeError: no valid NIGHT were selected!

In [9]:
print(ra_dec)

(264.79014464874683, 33.735652626899395)


In [5]:
print(current_filename)

/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1920/20210611/spectra-9-1920-thru20210611.fits


In [6]:
debug_spectra = read_spectra(current_filename)

INFO:spectra.py:291:read_spectra: iotime 0.617 sec to read spectra-9-1920-thru20210611.fits at 2022-07-07T11:25:57.412297


In [7]:
debug_spectra.fibermap

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKID,BRICK_OBJID,MORPHTYPE,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,BRICKNAME,EBV,FLUX_W1,FLUX_W2,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,PLATE_RA,PLATE_DEC,NUM_ITER,FIBER_X,FIBER_Y,DELTA_X,DELTA_Y,FIBER_RA,FIBER_DEC,EXPTIME,PSF_TO_FIBER_SPECFLUX,NIGHT,EXPID,MJD,TILEID
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int32,float64,int32,int16,int32,int32,bytes4,float32,float32,float32,float32,float32,float32,int16,int64,bytes2,float32,float32,float32,float32,bytes8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float64,int32
39632957064875447,9,403,9403,4500,0,266.2687501236815,33.7946544335606,0.0,0.0,2015.5,5400.0,4611686018427387904,1,TGT,-228.6573,-273.61987,3400,0.3219297830016805,63,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,3400,4,4611686018427387904,0,0,34359738368,266.2687501236815,33.7946544335606,2,-228.65,-273.624,-0.007,0.002,266.2687170185111,33.79464627925847,648.2016,0.7925361064786447,20210610,93280,59376.39914365,1920
39632957064872753,9,452,9452,4501,0,266.1386533949549,33.661151058898064,0.0,0.0,2015.5,5400.0,257,1,TGT,-202.33998,-308.17804,3200,0.642478214383338,1,9011,514973,817,SER,0.8862551,5.3569055,18.835625,289.08862,81.03721,31.432491,0,0,,0.0,0.0,0.0,0.0,2662p337,0.046279516,50.544174,32.849037,2.8957403,1.0029893,0.36966568,2.2344177,7.8565235,0.3696732,2.2344327,7.8565574,6.0,0.82118344,0.0864078,0.058329042,N,3200,2,257,0,0,0,266.1386533949549,33.661151058898064,2,-202.332,-308.185,-0.008,0.003,266.138615366032,33.66113911160532,648.2016,0.7514857903073631,20210610,93280,59376.39914365,1920
39632957064875879,9,426,9426,4502,0,266.2914083381407,33.75373924036178,0.0,0.0,2015.5,5400.0,2658,1,TGT,-234.02277,-284.4401,3200,0.222864391505616,1,9011,514973,3943,PSF,0.31904078,0.29987398,0.6534685,664.4255,223.18588,116.0903,0,0,,0.0,0.0,0.0,0.0,2662p337,0.043540917,4.9941726,4.1948338,5.0209756,1.3566833,0.24803375,0.23313278,0.5080298,0.24803375,0.23313278,0.5080298,0.0,0.0,0.0,0.0,N,3200,2,2658,0,0,0,266.2914083381407,33.75373924036178,2,-234.017,-284.442,-0.005,-0.002,266.29138551738515,33.753746488269776,648.2016,0.7889999999999998,20210610,93280,59376.39914365,1920
39632962085456409,9,299,9299,4503,0,266.20724993994753,33.96934542220569,0.0,0.0,2015.5,5400.0,2658,1,TGT,-213.66554,-228.19244,3200,0.1939481219929432,1,9011,516170,2585,PSF,0.33525842,0.225162,0.42361772,896.28467,231.03795,145.54475,0,0,,0.0,0.0,0.0,0.0,2662p340,0.047461707,-1.0470843,-0.79511315,5.1871724,1.4417421,0.2609769,0.175274,0.3297589,0.2609769,0.175274,0.3297589,0.0,0.0,0.0,0.0,N,3200,2,2658,0,0,0,266.20724993994753,33.96934542220569,2,-213.655,-228.194,-0.007,0.003,266.2072163053154,33.96933337991767,648.2016,0.7889999999999998,20210610,93280,59376.39914365,1920
-9647739,9,339,9339,4504,2,266.2464976124884,33.89239460170474,0.0,0.0,0.0,5400.0,0,4,SKY,-222.79297,-248.12921,0,0.0,0,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0,0,,0.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,0,0,0,0,0,266.2464976124884,33.89239460170474,2,-222.809,-248.126,0.016,-0.001,266.24657299761867,33.89239938624722,648.2016,0.7934394390325233,20210610,93280,59376.39914365,1920
396329620